<a href="https://colab.research.google.com/github/pablobots/private_test/blob/main/Pediatric_Bone_Age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q kaggle

In [3]:
!mkdir ~/.kaggle 

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets list

ref                                                            title                                                size  lastUpdated          downloadCount  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
utkarshxy/who-worldhealth-statistics-2020-complete             World Health 2020 🌏 | For Geospatial Analysis         1MB  2021-01-04 18:28:24            530  
gpreda/pfizer-vaccine-tweets                                   Pfizer Vaccine Tweets                               373KB  2021-01-04 19:29:43            355  
google/android-smartphones-high-accuracy-datasets              Android smartphones high accuracy GNSS datasets       1GB  2020-12-23 01:51:11             79  
ashkhagan/women-representation-in-city-property-sanfrancisco   Women Representation in City Property SanFrancisco    3KB  2020-12-13 05:18:14             80  
arashnic/covid19-case-surveillance-public-use-

In [11]:
!kaggle datasets download -d kmader/rsna-bone-age -p ./

100% 9.28G/9.29G [02:25<00:00, 44.1MB/s]
100% 9.29G/9.29G [02:25<00:00, 68.3MB/s]


In [ ]:
!unzip rsna-bone-age.zip